In [ ]:
import itertools
import os
from datetime import timedelta

import pandas as pd
import matplotlib
import matplotlib.pyplot as plt
import seaborn as sns

sns.set_theme()
sns.set(context="notebook", font="Verdana", font_scale=1.5)
matplotlib.rcParams['figure.figsize'] = (16, 6)
matplotlib.rcParams['figure.dpi'] = 200
matplotlib.rcParams['lines.linewidth'] = 2.5

print("Default figure size:", matplotlib.rcParams['figure.figsize'])

In [ ]:
def calculate_synthetic_values(df: pd.DataFrame) -> pd.DataFrame:
    df["Date"] = pd.to_datetime("2022-07-15") + pd.to_timedelta(df.index.get_level_values("hour"), unit="h") + pd.to_timedelta(df.index.get_level_values("minute"), unit="m")
    
    return df

In [ ]:
# Print current directory
print(os.getcwd())

solar = pd.read_csv("./2022_summer_solar.csv", index_col=["hour", "minute"])
wind_offshore = pd.read_csv("./2022_summer_wind_offshore.csv", index_col=["hour", "minute"])
wind_onshore = pd.read_csv("./2022_summer_wind_onshore.csv", index_col=["hour", "minute"])


solar.rename(columns={"median": "Solar"}, inplace=True)
wind_offshore.rename(columns={"median": "Wind Offshore"}, inplace=True)
wind_onshore.rename(columns={"median": "Wind Onshore"}, inplace=True)

df = pd.DataFrame(data={
    "Solar": solar["Solar"],
    "Wind Offshore": wind_offshore["Wind Offshore"],
    "Wind Onshore": wind_onshore["Wind Onshore"],
}, index=solar.index)

calculate_synthetic_values(df)
df.set_index("Date", inplace=True)

squeezed_timeframe_duration = timedelta(minutes=10)
squeezed_timeframe_start = pd.to_datetime("2023-11-15 09:00:00")
squeezed_timeframe_end = squeezed_timeframe_start + squeezed_timeframe_duration
squeezed_timeframe = pd.date_range(start=squeezed_timeframe_start, end=squeezed_timeframe_end, periods=len(df))
# squeezed_timeframe = squeezed_timeframe[:-1]

squeezed_df = df.copy()
squeezed_df.index = squeezed_timeframe

df

In [ ]:
fig, axs = plt.subplots(1, 2, sharex=False, sharey=True)

palette = itertools.cycle(sns.color_palette())

lines = [
    {"label": "Solar", "color": next(palette)},
    {"label": "Wind Offshore", "color": next(palette)},
    {"label": "Wind Onshore", "color": next(palette)},
]

for line in lines:
    sns.lineplot(data=df, x=df.index, y=line["label"], label=line["label"], color=line["color"], ax=axs[0])

    
axs[0].set_title("Original data\n(median generation during summer 2022)")

xticks = df.index[::16]
xtick_labels = [x.strftime("%H:%M:%S") for x in xticks]
axs[0].set_xticks(xticks)
axs[0].set_xticklabels(xtick_labels, rotation=40, ha="right")

for line in lines:
    sns.lineplot(data=squeezed_df, x=squeezed_df.index, y=line["label"], label=line["label"], color=line["color"], ax=axs[1])
    
axs[1].set_title("Squeezed timeframe data\n(mapped to 2023-11-26 09:00-09:10)")

vlines = [
    {"label": "Start", "color": next(palette)},
    {"label": "End", "color": next(palette)},
]

for _ in range(10):
    print(next(palette))

for ax in axs:
    ax.axvline(ax.get_xticks()[0], ls='--', label=vlines[0]["label"], color=vlines[0]["color"])
    ax.axvline(ax.get_xticks()[-1], ls='--', label=vlines[1]["label"], color=vlines[1]["color"])

for ax in axs:
    ax.set_ylabel("Generation (MW)")
    ax.set_xlabel("Time")
    
xticks = squeezed_df.index[::16]
xtick_labels = [x.strftime("%H:%M:%S") for x in xticks]
axs[1].set_xticks(xticks)
axs[1].set_xticklabels(xtick_labels, rotation=40, ha="right")